In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from functions import main, save_df, import_to_sql
from graphics import *

df_all = main("https://www.cheapshark.com/api/1.0/deals")

In [6]:
df_games = df_all[['gameID', 'title', 'release_date', 'scores']].copy()
df_games.drop_duplicates(subset=['gameID'], inplace=True)
df_games = df_games.reset_index(drop=True)
save_df(df_games, "games")

In [7]:
df_shops = df_all[['store_id', 'store_name']].copy()
df_shops.drop_duplicates(subset=['store_id'], inplace=True)
df_shops.reset_index(drop=True)
save_df(df_shops, "shops")

In [8]:
df_gameunits = df_all[['gameID', 'store_id', 'original_price', 'discount_price']].copy()
save_df(df_gameunits, "units")

In [9]:
df_games_genres = df_all[['gameID', 'genre_name']].copy().explode("genre_name")

df_games_genres.insert(0, 'id', range(1, len(df_games_genres) + 1))


save_df(df_games_genres, "games_genres")

In [10]:
df_genres = df_all[['genre_name']].copy().explode('genre_name').drop_duplicates().reset_index(drop=True)
df_genres = df_genres[df_genres['genre_name'].notna()]

df_genres['description'] = 'null'

save_df(df_genres, "genres")

In [11]:
# SEND TO SQL

import_to_sql(df_games, 'games')

import_to_sql(df_genres, 'genres')

import_to_sql(df_games_genres, 'games_genres')

import_to_sql(df_shops, 'shops')

import_to_sql(df_gameunits, 'units')


In [18]:
# RECEIVE FROM SQL

import pandas as pd
import pymysql
from sqlalchemy import create_engine
import os
import getpass  # Para obtener la contraseña sin mostrar la entrada

# Obtener la contraseña de forma segura
password = getpass.getpass()

# Configuración de la base de datos y la conexión
bd = os.getenv('bbdd_name')
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

# Leer la consulta SQL en un DataFrame
consulta = "SELECT * FROM games WHERE gameID = 170045" #AQUI PEGAR LA QUERY SIN ;  EN UN SOLO BLOQUE 
df_query = pd.read_sql(consulta, con=engine)

In [13]:
#average_disc_vs_shop_bar(deals_df)
# min_max_disc_vs_shop_bar(deals_df)
# linear_graph_discounts_vs_punct_linear(deals_df)
# genre_vs_mean_discount_bar(unroll_list_from_dfcolumn(deals_df, 'genre'))
# circular_pie_chart_genres(unroll_list_from_dfcolumn(deals_df, 'genre'))
# linear_graph_discounts_vs_releasedate(clean_df_releases(deals_df))

In [14]:
# streamlit run app.py